# Scraping data from HANDSHAKE

In [33]:
import requests
import pandas as pd
import numpy as np
import re
import lxml.html as lx
import time
from bs4 import BeautifulSoup
import requests_cache
requests_cache.install_cache("fp")
from tqdm import tqdm
import string
import nltk
import warnings
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
driver_path = " /Users/masheng/Library/Python/3.11/lib/python/site-packages"

options = webdriver.ChromeOptions()
options.add_argument(driver_path)

driver = webdriver.Chrome(options=options)

In [5]:
url = "https://ucdavis.joinhandshake.com/stu/postings"
driver.get(url)

### tests

In [60]:
# find all jobs_bar in this page
job_list = driver.find_element("xpath", '//*[@id="skip-to-content"]/div[4]/div/div[1]/div/div/form/div[2]/div/div/div[1]/div[3]/div/div[1]')

In [30]:
a_elements = job_list.find_element(By.TAG_NAME, 'a')

In [31]:
# for job_bar in a_elements:
job_bar.click()

In [62]:
# more information
more_buttom = driver.find_element("xpath", '//*[@id="skip-to-content"]/div[4]/div/div[1]/div/div/form/div[2]/div/div/div[2]/div[1]/div[3]/div[4]/div/div/div/button')
more_buttom.click()

In [77]:
more_buttom = driver.find_element("xpath", "//button[contains(text(), 'More')]")
more_buttom.click()

In [42]:
# job_detail = driver.find_element("xpath", '//*[@id="skip-to-content"]/div[4]/div/div[1]/div/div/form/div[2]/div/div/div[2]/div[1]')
job_detail.text

"Nexon America\nOther Industries\nData Analyst Intern\nPosted 4 days ago∙Apply by March 25, 2024 at 6 AM\nSave\nApply Externally\nAt a glance\n$20–22/hr\nHybrid, based in El Segundo, CA\nWork in person for part of the week, from the location\nInternship\nFull-time∙From June 10 to September 2\nUS work authorization required\nOpen to candidates with OPT/CPT\nAbout Nexon\nNexon America is a branch of Nexon Co., Ltd., a global video game publisher and leading developer of online virtual worlds for PCs, consoles, and mobile devices. Nexon proudly offers celebrated franchises like MapleStory, KartRider, and Mabinogi that have been enjoyed by millions of players for over two decades. When we founded our Nexon America branch in 2006, we made it our goal to bring those unique games to western players using a player-first approach. We achieve that goal by applying our team’s outstanding free-to-play expertise and live game support to every decision, every day.\nSummary of Position\nNexon America

In [41]:
last_button = driver.find_element('xpath','//*[@id="skip-to-content"]/div[4]/div/div[1]/div/div/form/div[2]/div/div/div[1]/div[3]/div/div[2]/div/div/button[1]')

In [48]:
next = next_button.get_attribute('disabled')
print(next)

None


In [68]:
next_button = driver.find_element('xpath','//*[@id="skip-to-content"]/div[4]/div/div[1]/div/div/form/div[2]/div/div/div[1]/div[3]/div/div[2]/div/div/button[2]')
next_button.get_attribute('disabled')

In [70]:
np.random.normal(loc=0, scale=1, size=1)[0]

1.3243228547953614

## scrape the job information

In [78]:
all_data_analysts_json = {}  # create a dictionary to store all data analysts' information
i = 0   # index of all_data_analysts_json
page_i = 0  # index of jobs in this page
page = 1    # index of page

# find all jobs_bar in this page
job_list = driver.find_element("xpath", '//div[@class="style__cards___gaXdn"]')
a_elements = job_list.find_elements(By.TAG_NAME, 'a')

for job_bar in a_elements:

    # SET RANDOM TIME
    while True:
        random_num = np.random.normal(5, 1)
        if 5 < random_num <= 10:
            break
    time.sleep(random_num)
    job_bar.click()
    while True:
        random_num = np.random.normal(2, 1)
        if 2 < random_num <= 5:
            break
    time.sleep(random_num)

    # more information
    more_buttom = driver.find_element("xpath", "//button[contains(text(), 'More')]")
    while True:
        random_num = np.random.normal(2, 1)
        if 2 < random_num <= 5:
            break
    time.sleep(random_num)
    if more_buttom:
        more_buttom.click()
    
    # get job detail
    job_detail_tag = driver.find_element("xpath", '//div[@class="style__details-paddingV2___FdddG"]')
    job_detail = info = BeautifulSoup(job_detail_tag.text, 'html.parser')
    all_data_analysts_json[i] = job_detail
    page_i = page_i + 1
    i = i+1

print(f'page {page}: {page_i+1} - total: {i+1}')
next_button = driver.find_element('xpath','//button[@data-hook="search-pagination-next"]')

# loop through all pages
while next_button.get_attribute('disabled') and page<2:
    next_button.click()
    page = page+1
    page_i = 0
    job_list = driver.find_element("xpath", '//div[@class="style__cards___gaXdn"]')
    a_elements = job_list.find_elements(By.TAG_NAME, 'a')

    for job_bar in a_elements:
        while True:
            random_num = np.random.normal(5, 1)
            if 0 < random_num <= 10:
                break
        time.sleep(random_num)
        job_bar.click()
        more_buttom = driver.find_element("xpath", "//button[contains(text(), 'More')]")
        if more_buttom:
            more_buttom.click()
        job_detail_tag = driver.find_element("xpath", '//div[@class="style__details-paddingV2___FdddG"]')
        job_detail = info = BeautifulSoup(job_detail_tag.text, 'html.parser')
        all_data_analysts_json[i] = job_detail
        page_i = page_i + 1
        i = i+1
    print(f'page {page}: {page_i+1} - total: {i+1}')
    next_button = driver.find_element('xpath','//button[@data-hook="search-pagination-next"]')

print(f"Finish scraping!")

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=122.0.6261.112); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
0   chromedriver                        0x0000000104800508 chromedriver + 3966216
1   chromedriver                        0x00000001047f8a94 chromedriver + 3934868
2   chromedriver                        0x000000010447bda0 chromedriver + 277920
3   chromedriver                        0x0000000104480648 chromedriver + 296520
4   chromedriver                        0x0000000104481fe0 chromedriver + 303072
5   chromedriver                        0x0000000104482078 chromedriver + 303224
6   chromedriver                        0x00000001044c3da4 chromedriver + 572836
7   chromedriver                        0x00000001044c2044 chromedriver + 565316
8   chromedriver                        0x00000001044c